In [1]:
#Enable matplotlib to display in jupyter notebook & import it
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import re

In [ ]:
#Check working directory and move it to the folder where I want to work
#%pwd
%ls
%cd BOSTONAIRBB

In [3]:
#read files
listings = pd.read_csv('data/listings.csv')
calendar = pd.read_csv('data/calendar.csv') #NOTE I changed the British for the 'Merican pronunciation
reviews = pd.read_csv('data/reviews.csv')

In [3]:
#Rob's Read file names
listings = pd.read_csv('boston_listings.csv')
calendar = pd.read_csv('boston_calendar.csv') #NOTE I changed the British for the 'Merican pronunciation
reviews = pd.read_csv('boston_reviews.csv')

In [4]:
#Data manipulation steps to clean data for analysis

###################################      LISTINGS      ##################################################

#coerce appropriate fields to datetime
listings['host_since'] = pd.to_datetime(listings['host_since'])

listings['price'] = listings['price'].str.replace('$','')
listings['price'] = listings['price'].str.replace(',','')
listings['price'] = listings['price'].astype(float)

listings['zipcode'] = listings['zipcode'].str.replace('-','')
listings['zipcode'] = listings['zipcode'].str.replace(' ', '')
listings['mainzip'] = pd.Series([str(z)[:5] for z in listings['zipcode'].fillna('00000')]) ##THIS IS NOT WORKING AS I WANT IT TO; AIMING TO GRAB FIRST FIVE NUMBERS TO CREATE NEW FILE

#CLEANING FEE

listings['cleaning_fee'] = listings['cleaning_fee'].str.replace('$','')
listings['cleaning_fee'] = listings['cleaning_fee'].str.replace(',','')

listings['cleaning_fee'] = listings['cleaning_fee'].astype(float)

#CALENDAR LAST SCRAPED
listings['calendar_last_scraped'] = pd.to_datetime(listings['calendar_last_scraped'])


##################################      REVIEWS       #####################################################
reviews['date'] = pd.to_datetime(reviews['date'])

##################################      CALENDAR       #####################################################
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['listing_id'] = calendar['listing_id'].astype('category')
calendar['price'] = calendar['price'].str.replace('$','')
calendar['price'] = calendar['price'].str.replace(',','')
calendar['price'].fillna(0)
calendar['price'] = calendar['price'].astype(float)



In [19]:
###################################      LISTINGS      ##################################################
#Rob: Here I borrowed code from kaggle, cleaning up the junk in the amenities list, removing parentheses, etc. first
#Then I isolated each unique amenity and created column t/f data frames for them, and then eventually concatenated 
#with original listings data to build features. Just see the features df

#Dummy Variables in Listings
listings['amenities'] = listings['amenities'].map(
    lambda amns: "|".join([amn.replace("}", "").replace("{", "").replace('"', "")\
                           for amn in amns.split(",")]))
                           
amenities = np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|"))))[1:]
amenity_arr = np.array([listings['amenities'].map(lambda amns: amn in amns) for amn in amenities[0:43]])

#Listing exploratory
listings['amenities'].map(lambda amns: amns.split("|")).head()
np.unique(np.concatenate(listings['amenities'].map(lambda amns: amns.split("|"))))[1:]

#Merged dataset with listings and dummy variables
features = pd.concat([listings, pd.DataFrame(data=amenity_arr.T, columns=amenities[0:43])], axis=1)
features


id                            listing_url       scrape_id  \
0     12147973  https://www.airbnb.com/rooms/12147973  20160906204935   
1      3075044   https://www.airbnb.com/rooms/3075044  20160906204935   
2         6976      https://www.airbnb.com/rooms/6976  20160906204935   
3      1436513   https://www.airbnb.com/rooms/1436513  20160906204935   
4      7651065   https://www.airbnb.com/rooms/7651065  20160906204935   
5     12386020  https://www.airbnb.com/rooms/12386020  20160906204935   
6      5706985   https://www.airbnb.com/rooms/5706985  20160906204935   
7      2843445   https://www.airbnb.com/rooms/2843445  20160906204935   
8       753446    https://www.airbnb.com/rooms/753446  20160906204935   
9       849408    https://www.airbnb.com/rooms/849408  20160906204935   
10    12023024  https://www.airbnb.com/rooms/12023024  20160906204935   
11     1668313   https://www.airbnb.com/rooms/1668313  20160906204935   
12     2684840   https://www.airbnb.com/rooms/2684840  20160906204935   
13    13547301  https://www.airbnb.com/rooms/13547301  20160906204935   
14     5434353   https://www.airbnb.com/rooms/5434353  20160906204935   
15      225979    https://www.airbnb.com/rooms/225979  20160906204935   
16     3420384   https://www.airbnb.com/rooms/3420384  20160906204935   
17    13512930  https://www.airbnb.com/rooms/13512930  20160906204935   
18     7482195   https://www.airbnb.com/rooms/7482195  20160906204935   
19     7252607   https://www.airbnb.com/rooms/7252607  20160906204935   
20     2583074   https://www.airbnb.com/rooms/2583074  20160906204935   
21    13251243  https://www.airbnb.com/rooms/13251243  20160906204935   
22      225834    https://www.airbnb.com/rooms/225834  20160906204935   
23     6400432   https://www.airbnb.com/rooms/6400432  20160906204935   
24     5498472   https://www.airbnb.com/rooms/5498472  20160906204935   
25      894539    https://www.airbnb.com/rooms/894539  20160906204935   
26      879929    https://www.airbnb.com/rooms/879929  20160906204935   
27     9218312   https://www.airbnb.com/rooms/9218312  20160906204935   
28      321328    https://www.airbnb.com/rooms/321328  20160906204935   
29     1810172   https://www.airbnb.com/rooms/1810172  20160906204935   
...        ...                                    ...             ...   
3555  13015653  https://www.airbnb.com/rooms/13015653  20160906204935   
3556  14849141  https://www.airbnb.com/rooms/14849141  20160906204935   
3557  14604429  https://www.airbnb.com/rooms/14604429  20160906204935   
3558  14592644  https://www.airbnb.com/rooms/14592644  20160906204935   
3559  12915510  https://www.airbnb.com/rooms/12915510  20160906204935   
3560  14887382  https://www.airbnb.com/rooms/14887382  20160906204935   
3561  14823430  https://www.airbnb.com/rooms/14823430  20160906204935   
3562  14881840  https://www.airbnb.com/rooms/14881840  20160906204935   
3563  11452297  https://www.airbnb.com/rooms/11452297  20160906204935   
3564  14576706  https://www.airbnb.com/rooms/14576706  20160906204935   
3565  14335003  https://www.airbnb.com/rooms/14335003  20160906204935   
3566  14871445  https://www.airbnb.com/rooms/14871445  20160906204935   
3567  14472678  https://www.airbnb.com/rooms/14472678  20160906204935   
3568  14574800  https://www.airbnb.com/rooms/14574800  20160906204935   
3569  14426586  https://www.airbnb.com/rooms/14426586  20160906204935   
3570   3131519   https://www.airbnb.com/rooms/3131519  20160906204935   
3571  14592547  https://www.airbnb.com/rooms/14592547  20160906204935   
3572  14744740  https://www.airbnb.com/rooms/14744740  20160906204935   
3573  14504583  https://www.airbnb.com/rooms/14504583  20160906204935   
3574  14743129  https://www.airbnb.com/rooms/14743129  20160906204935   
3575   5280827   https://www.airbnb.com/rooms/5280827  20160906204935   
3576  14689681  https://www.airbnb.com/rooms/14689681  20160906204935   
3577  13750763  https://www.airbnb.com/rooms/13750763  20160906204935 

In [7]:
#Rob: groupby'ed the calendar listing IDs for total rev, count of bookings/bookigns, and then revenue / booking
###########################      CALENDAR SUMMARY FOR MERGING       #########################################
cleanCal = calendar[calendar['available'] == 't']
cleanCal = calendar.groupby('listing_id', as_index = False)['price'].sum().sort_values(['price'], ascending = False)
cleanCal.columns = ['listing_id', 'total_revenue']
cleanCal2 = calendar.groupby('listing_id', as_index = False)['price'].count().sort_values(['price'], ascending = False)
cleanCal2.columns = ['listing_id', 'num_bookings']
cleanCal = cleanCal.merge(cleanCal2)
cleanCal['total_revenue'] = cleanCal['total_revenue'].fillna(0)
cleanCal['revenue_per_booking'] = cleanCal['total_revenue']/cleanCal['num_bookings']
cleanCal['revenue_per_booking'] = cleanCal['revenue_per_booking'].fillna(0)
cleanCal

listing_id  total_revenue  num_bookings  revenue_per_booking
0       12972378      1344000.0           336          4000.000000
1        8303267       574300.0           333          1724.624625
2        6627802       365000.0           365          1000.000000
3       12184433       364635.0           365           999.000000
4       14798673       363150.0           270          1345.000000
5        6838488       339550.0           357           951.120448
6       14758045       333450.0           270          1235.000000
7        6066455       290984.0           331           879.105740
8        7633883       280158.0           279          1004.150538
9        1810397       267200.0           334           800.000000
10       2881388       265044.0           327           810.532110
11      10036192       259994.0           365           712.312329
12       7740592       256700.0           364           705.219780
13       3673688       255135.0           365           699.000000
14      13606268       251871.0           365           690.057534
15      11970702       249900.0           357           700.000000
16         50032       238525.0           329           725.000000
17      11521541       237320.0           349           680.000000
18       7134081       235328.0           337           698.302671
19       6544715       233850.0           336           695.982143
20       6326257       228338.0           340           671.582353
21       7611271       227239.0           354           641.918079
22       9231486       222013.0           342           649.160819
23       6629066       213100.0           331           643.806647
24       5690424       209990.0           301           697.641196
25       2277821       209386.0           344           608.680233
26      14463855       207046.0           364           568.807692
27       4812786       205970.0           365           564.301370
28      14035936       205800.0           343           600.000000
29       5783197       204500.0           331           617.824773
...          ...            ...           ...                  ...
3555    14122467            0.0             0             0.000000
3556    14147343            0.0             0             0.000000
3557    14161407            0.0             0             0.000000
3558    14218904            0.0             0             0.000000
3559    14221294            0.0             0             0.000000
3560    14246616            0.0             0             0.000000
3561    14250499            0.0             0             0.000000
3562    14252020            0.0             0             0.000000
3563    14335003            0.0             0             0.000000
3564    14367134            0.0             0             0.000000
3565    14443295            0.0             0             0.000000
3566    14460344            0.0             0             0.000000
3567    14472678            0.0             0             0.000000
3568    14472812            0.0             0             0.000000
3569    14473036            0.0             0             0.000000
3570    14473053            0.0             0             0.000000
3571    14473564            0.0             0             0.000000
3572    14506396            0.0             0             0.000000
3573    14565726            0.0             0             0.000000
3574    14572515            0.0             0             0.000000
3575    14592547            0.0             0             0.000000
3576    14651378            0.0             0             0.000000
3577    14668008            0.0             0             0.000000
3578    14668441            0.0             0             0.000000
3579    14671948            0.0             0             0.000000
3580    14673441            0.0             0             0.000000
3581    14722711            0.0             0             0.000000
3582    14774868            0.0             0        

In [11]:
len(cleanCal.revenue_per_booking)

3585

In [10]:
cleanCal.revenue_per_booking.value_counts().sort_index()

0.000000       679
11.000000        1
20.000000        1
25.000000        1
27.000000        1
28.410959        1
29.000000        1
29.424658        1
29.871233        1
30.383562        1
31.000000        6
31.104938        1
31.296188        1
33.000000        2
33.252055        1
33.808824        1
33.870370        1
34.000000        1
34.355263        1
35.000000        8
35.028986        1
35.101370        1
35.204819        1
35.215947        1
36.000000        2
36.562842        1
37.265217        1
37.594470        1
38.000000        2
38.435484        1
              ... 
648.548387       1
649.160819       1
652.868966       1
653.265306       1
669.246032       1
671.582353       1
680.000000       1
690.057534       1
695.982143       1
697.641196       1
698.302671       1
699.000000       1
700.000000       1
705.219780       1
712.312329       1
725.000000       1
742.647059       1
800.000000       1
810.532110       1
819.111111       1
879.105740       1
951.120448  

In [8]:
#Rob
#Initial Master Merging Merging by IDs between cal and "features" df
listingRev = features.merge(cleanCal, how = 'left', left_on='id', right_on='listing_id')
listingRev

id                            listing_url       scrape_id  \
0     12147973  https://www.airbnb.com/rooms/12147973  20160906204935   
1      3075044   https://www.airbnb.com/rooms/3075044  20160906204935   
2         6976      https://www.airbnb.com/rooms/6976  20160906204935   
3      1436513   https://www.airbnb.com/rooms/1436513  20160906204935   
4      7651065   https://www.airbnb.com/rooms/7651065  20160906204935   
5     12386020  https://www.airbnb.com/rooms/12386020  20160906204935   
6      5706985   https://www.airbnb.com/rooms/5706985  20160906204935   
7      2843445   https://www.airbnb.com/rooms/2843445  20160906204935   
8       753446    https://www.airbnb.com/rooms/753446  20160906204935   
9       849408    https://www.airbnb.com/rooms/849408  20160906204935   
10    12023024  https://www.airbnb.com/rooms/12023024  20160906204935   
11     1668313   https://www.airbnb.com/rooms/1668313  20160906204935   
12     2684840   https://www.airbnb.com/rooms/2684840  20160906204935   
13    13547301  https://www.airbnb.com/rooms/13547301  20160906204935   
14     5434353   https://www.airbnb.com/rooms/5434353  20160906204935   
15      225979    https://www.airbnb.com/rooms/225979  20160906204935   
16     3420384   https://www.airbnb.com/rooms/3420384  20160906204935   
17    13512930  https://www.airbnb.com/rooms/13512930  20160906204935   
18     7482195   https://www.airbnb.com/rooms/7482195  20160906204935   
19     7252607   https://www.airbnb.com/rooms/7252607  20160906204935   
20     2583074   https://www.airbnb.com/rooms/2583074  20160906204935   
21    13251243  https://www.airbnb.com/rooms/13251243  20160906204935   
22      225834    https://www.airbnb.com/rooms/225834  20160906204935   
23     6400432   https://www.airbnb.com/rooms/6400432  20160906204935   
24     5498472   https://www.airbnb.com/rooms/5498472  20160906204935   
25      894539    https://www.airbnb.com/rooms/894539  20160906204935   
26      879929    https://www.airbnb.com/rooms/879929  20160906204935   
27     9218312   https://www.airbnb.com/rooms/9218312  20160906204935   
28      321328    https://www.airbnb.com/rooms/321328  20160906204935   
29     1810172   https://www.airbnb.com/rooms/1810172  20160906204935   
...        ...                                    ...             ...   
3555  13015653  https://www.airbnb.com/rooms/13015653  20160906204935   
3556  14849141  https://www.airbnb.com/rooms/14849141  20160906204935   
3557  14604429  https://www.airbnb.com/rooms/14604429  20160906204935   
3558  14592644  https://www.airbnb.com/rooms/14592644  20160906204935   
3559  12915510  https://www.airbnb.com/rooms/12915510  20160906204935   
3560  14887382  https://www.airbnb.com/rooms/14887382  20160906204935   
3561  14823430  https://www.airbnb.com/rooms/14823430  20160906204935   
3562  14881840  https://www.airbnb.com/rooms/14881840  20160906204935   
3563  11452297  https://www.airbnb.com/rooms/11452297  20160906204935   
3564  14576706  https://www.airbnb.com/rooms/14576706  20160906204935   
3565  14335003  https://www.airbnb.com/rooms/14335003  20160906204935   
3566  14871445  https://www.airbnb.com/rooms/14871445  20160906204935   
3567  14472678  https://www.airbnb.com/rooms/14472678  20160906204935   
3568  14574800  https://www.airbnb.com/rooms/14574800  20160906204935   
3569  14426586  https://www.airbnb.com/rooms/14426586  20160906204935   
3570   3131519   https://www.airbnb.com/rooms/3131519  20160906204935   
3571  14592547  https://www.airbnb.com/rooms/14592547  20160906204935   
3572  14744740  https://www.airbnb.com/rooms/14744740  20160906204935   
3573  14504583  https://www.airbnb.com/rooms/14504583  20160906204935   
3574  14743129  https://www.airbnb.com/rooms/14743129  20160906204935   
3575   5280827   https://www.airbnb.com/rooms/5280827  20160906204935   
3576  14689681  https://www.airbnb.com/rooms/14689681  20160906204935   
3577  13750763  https://www.airbnb.com/rooms/13750763  20160906204935 

In [20]:
#Neighborhood Summary
#The super groupby by neighborhood. Collin feel free to replace neighbourhood_cleased with mainzip for zip

#The valuable columns I chose were the revenue metrics, # of reviews, beds, baths, review scores, and then amenities
nbr1 = listingRev.groupby('neighbourhood_cleansed', as_index = False)['total_revenue', 'num_bookings', 'number_of_reviews'].sum().sort_values(['total_revenue'])
nbr2 = listingRev.groupby('neighbourhood_cleansed', as_index = False)['accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'review_scores_accuracy','review_scores_cleanliness', 
         'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', ].mean().sort_values(['review_scores_value'])

amenityList = ['24-Hour Check-in', 'Air Conditioning', 'Breakfast', 'Buzzer/Wireless Intercom', 'Cable TV', 'Carbon Monoxide Detector','Cat(s)', 'Dog(s)', 'Doorman','Essentials', 'Family/Kid Friendly', 'Fire Extinguisher', 'First Aid Kit', 'Free Parking on Premises', 'Free Parking on Street', 'Gym', 'Hair Dryer', 'Heating', 'Hot Tub','Indoor Fireplace', 'Internet', 'Iron', 'Kitchen','Laptop Friendly Workspace', 'Lock on Bedroom Door', 'Other pet(s)','Paid Parking Off Premises', 'Pets Allowed','Pets live on this property', 'Pool','Smoking Allowed', 'Suitable for Events', 'TV','Washer / Dryer', 'Wheelchair Accessible','Wireless Internet']

dummy1 = listingRev.groupby('neighbourhood_cleansed', as_index = False)[amenityList].sum()
dummy2 = listingRev.groupby('neighbourhood_cleansed', as_index = False)[amenityList].count()                                                              
dummy = dummy1[list(range(1,36))]/dummy2[list(range(1,36))]
dummy['neighbourhood_cleansed'] = dummy1[[0]]

#Super Merge
#Here's the final summary

nbhdsummary = nbr1
for df in [nbr2, nbr3, dummy]:
    nbhdsummary = nbhdsummary.merge(df)
    
nbhdsummary.sort_values(['total_revenue'], ascending = False)
nbhdsummary

neighbourhood_cleansed  total_revenue  num_bookings  number_of_reviews  \
0     Longwood Medical Area       202758.0          1690                393   
1          Leather District       249895.0           725                 69   
2                 Hyde Park       394644.0          5732                376   
3                  Mattapan       526421.0          6702                442   
4              West Roxbury      1197746.0         10281                690   
5                Roslindale      1287744.0         13541               1387   
6               Bay Village      1761264.0          5732                241   
7              Mission Hill      2464145.0         16138               1469   
8                  West End      3416581.0         10880                157   
9               East Boston      4027926.0         29291               5504   
10                Chinatown      4116480.0         13037                669   
11              Charlestown      4291422.0         21272               1981   
12                 Brighton      4295955.0         31199               3052   
13  South Boston Waterfront      4310096.0         13377                744   
14                  Allston      4499253.0         34293               2988   
15                  Roxbury      4840368.0         29756               3775   
16                North End      5379494.0         23581               4483   
17               Dorchester      5632963.0         63587               7396   
18             South Boston      7037234.0         32981               3851   
19              Beacon Hill      8261270.0         31753               4098   
20                 Downtown      8513860.0         31243               2535   
21            Jamaica Plain      9531622.0         61742               9055   
22                   Fenway     12464306.0         43363               2817   
23                South End     12656093.0         54071               5714   
24                 Back Bay     16244021.0         57070               4389   

    accommodates  bathrooms  bedrooms      beds  review_scores_rating  \
0       2.666667   1.111111  1.111111  1.750000             94.000000   
1       2.800000   1.200000  1.400000  1.800000             98.333333   
2       2.483871   1.193548  1.354839  1.419355             92.280000   
3       2.500000   1.229167  1.041667  1.625000             91.000000   
4       3.000000   1.434783  1.347826  1.782609             95.212121   
5       3.071429   1.241071  1.267857  1.696429             95.380000   
6       3.541667   1.458333  1.458333  1.833333             93.000000   
7       2.435484   1.193548  1.161290  1.487805             87.695122   
8       3.857143   1.459184  1.530612  1.836735             90.333333   
9       2.906667   1.163333  1.260000  1.606667             90.272059   
10      3.732394   1.316901  1.450704  1.732394             92.121951   
11      3.207207   1.256757  1.423423  1.702703             93.697368   
12      2.740541   1.178378  1.243243  1.572973             90.864662   
13      3.686747   1.445783  1.373494  1.807229             94.431034   
14      2.557692   1.226923  1.173077  1.419231             90.164835   
15      3.131944   1.230216  1.263889  1.744681             89.803419   
16      3.391608   1.124113  1.321678  1.830986             93.725000   
17      2.617100   1.198502  1.315985  1.561798             89.280000   
18      3.488506   1.445402  1.465517  1.827586             93.980392   
19      2.850515   1.113402  1.072539  1.484536             93.378698   
20      3.639535   1.247093  1.232558  1.697674             92.432203   
21      3.198251   1.219298  1.483965  1.749271             94.159322   
22      2.968966   1.229167  1.148276  1.519031             89.417112   
23      2.886503   1.141975  1.084375  1.432515             93.264925   
24      3.251656   1.168874  1.110368  1.536424             91.531646   

    review_scores_accuracy          ...            

In [ ]:
#understand the datatypes explicitly in the dataset
listings.dtypes
calendar.dtypes
reviews.dtypes


#check column names .columns

#check headers .head()

#check tails .tail()

#listings.columns
listings.tail()
#reviews.head()
#listings['neighborhood_overview'].head(5)


In [ ]:
listings_column_names = pd.DataFrame((listings.columns))
listings_column_names_list = list(listings.columns)
object_first_observation = [listings[x].head(1) for x in listings_column_names_list]
observations = pd.DataFrame(object_first_observation)

In [ ]:
observations.to_csv('datasample.csv')

In [ ]:
reviews['listing_id'].value_counts().plot(kind='hist', figsize = (20,10), fontsize = 16, title = 'Frequency of Reviews')
axes = plt.gca()
axes.set_xlim([0,250])
axes.set_ylim([0,2500])
plt.savefig('freq reviews.jpg')
#multi_reviews = (reviews['listing_id'].value_counts() > 1)
#multi_reviews.value_counts().plot(kind='bar')
#data = go.Histogram(x = multi_reviews)
#py.iplot(data, filename = 'reviewsmorethan1')

In [ ]:
#listings['zipcode'].value_counts().plot(kind = 'bar')
zipcodes_listings = listings['mainzip'].value_counts()
zipcodes_listings.plot()

In [ ]:
#plt.scatter(x = listings['host_response_time'], y = listings['review_scores_value'])
listings['host_response_time'].value_counts()
listings['host_response_time_categories'] = np.nan
listings.loc[listings['host_response_time'] == 'within an hour', 'host_response_time_categories'] = 0
listings.loc[listings['host_response_time'] == 'within a few hours', 'host_response_time_categories'] = 1
listings.loc[listings['host_response_time'] == 'within a day', 'host_response_time_categories'] = 2
listings.loc[listings['host_response_time'] == 'a few days or more', 'host_response_time_categories'] = 3

#df.loc[df['Sex'] == 'M', 'Sex_int'] = 0
#df.loc[df['Sex'] == 'F', 'Sex_int'] = 1

In [ ]:
plt.scatter(x = listings['host_response_time_categories'], y = listings['review_scores_value'])

In [ ]:
#plt.scatter(x= listings['zipcode'],y= listings['price'])
#plt.figsize()
#plt.xlabel('zipcode') #,fontsize = )
#plt.ylabel('price')
#fig.savefig('scatter.jpg')
listings['host_response_time'].value_counts()

In [ ]:
#listings.corr().to_csv('listings_correlations.csv')

In [ ]:
split_amenities = listings['amenities'].str.split(',')
list_split_amenities = list(split_amenities)
new_list_split_amenities = list()
for i in list_split_amenities:
    for j in i:
        new_list_split_amenities.append(j)



In [ ]:
#len(new_list_split_amenities)
reduced_new_list_split_amenities = list(set(new_list_split_amenities))
len(reduced_new_list_split_amenities)

In [ ]:
#THIS CODE IS INTENDED TO FIND THE REVENUE 
listings_unique_ids = list(listings['id'].unique())
len(listings_unique_ids)


In [ ]:
#calendar.plot(kind ='panel')
#calendar.set_index = calendar['listing_id']

#sorted(calendar, key = lambda x: (x['listing_id'], x['date']))
calendar.groupby(['listing_id', 'date']).head()
calendar.sort_index(ascending = False)

#Capture median values
calendar[calendar['listing_id'] == 14504422]
count_for_id = calendar[(calendar['available'] == 'f') & (calendar['listing_id'] == 14504422)]['available'].count() 
mean_price =  calendar[(calendar['available'] == 't') & (calendar['listing_id'] == 14504422)]['price'].mean()

revenue_wo_cleaning = mean_price * count_for_id
revenue_wo_cleaning

In [ ]:
for i in 
count_for_id = calendar[(calendar['available'] == 'f') & (calendar['listing_id'] == 14504422)]['available'].count() 
mean_price =  calendar[(calendar['available'] == 't') & (calendar['listing_id'] == 14504422)]['price'].mean()